## 1 - Download image chips from Google Earth Engine

This notebook represents the initial step in the application of a methodology for identifying reforestation projects in the Brazilian Atlantic Rainforest. Developed by Felipe Begliomini as part of his MRes dissertation for the AI4ER program at the University of Cambridge, this notebook serves as the foundation for the project.

The primary objective of this notebook is to provide the necessary steps for downloading image chips from Google Earth Engine. The data will be used to train a U-Net model, enabling the identification of reforestation projects within the Brazilian Atlantic Rainforest. Inspiration for this notebook was drawn from two key sources: [Fast(er) Downloads](https://gorelick.medium.com/fast-er-downloads-a2abd512aa26) and the [geemap notebook](https://geemap.org/notebooks/96_image_chips/).

#### Import files

In [ ]:
#if you are running at colab or other internet-based env
!pip install earthengine-api
!pip install geemap
!pip install retry

In [2]:
import ee
from src.data.tools.download_gee import GetImageChips
import geemap

#### Authenticate and Initialize GEE to use the high-volume endpoint

- [high-volume endpoint](https://developers.google.com/earth-engine/cloud/highvolume)

In [3]:
# Uncoment the next line if you need to authinticate this device
#ee.Authenticate()

ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com')

#### Import the field data

In [7]:
# Import the rasterized data in raster format (5m)
ref_raster = ee.Image('users/fnincao/MRes/sudeste_data')

# Importe the reference data in poly format
ref_vector = ee.FeatureCollection('users/fnincao/MRes/polygons')

# Define the region of Interest
region = ref_raster.geometry()

#### Import the points that will be used to create the image chips

In [8]:
# Import the points for sample the region
points = ee.FeatureCollection('users/fnincao/MRes/points')\
           .filterBounds(region)
    
points

#### Generates the images to extract the chips

##### Reference Image

In [9]:
# Convert the reference image to a binary mask
ref_image = ref_raster\
        .where(ref_raster.gt(0), 1)\
        .unmask(0)\
        .toInt()

##### Planet Image (NIR / RED / GREEN) (5m)

In [10]:
# Create a composite from Planet basemap from Jan-Apr 2023
planet_image =  ee.ImageCollection("projects/planet-nicfi/assets/basemaps/americas")\
            .filterDate('2023-01-01','2023-04-01')\
            .median()\
            .select('N','R','G')


#### Sentinel-1 Images (Band-C VV and VH) (10m)

In [ ]:
s1_image = ee.ImageCollection('COPERNICUS/S1_GRD')\
            .filterDate('2020-06-01', '2020-10-01')\
            .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))\
            .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))\
            .filter(ee.Filter.eq('instrumentMode', 'IW'))\
            .filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))\
            .select('VH').mean()


#### Sentinel-2 Images (Red-Edge) (20m)

In [ ]:
sentinel_image_2022 = ee.ImageCollection('COPERNICUS/S2_SR') \
                        .filterBounds(region) \
                        .filterDate('2022-05-01', '2022-09-01') \
                        .select('B5') \
                        .median() 
                   

sentinel_image_2021 = ee.ImageCollection('COPERNICUS/S2_SR') \
                        .filterBounds(region) \
                        .filterDate('2021-05-01', '2021-09-01') \
                        .select('B5') \
                        .median()

sentinel_image_2020 = ee.ImageCollection('COPERNICUS/S2_SR') \
                        .filterBounds(region) \
                        .filterDate('2020-05-01', '2020-09-01') \
                        .select('B5') \
                        .median()

sentinel_image = sentinel_image_2020.addBands([sentinel_image_2021,
                                               sentinel_image_2022])

#### Download the Reference images

In [11]:
%%time
GetImageChips(download_image=ref_image,
              out_resolution=5,
              points=points,
              out_dir='../../data/images/1_reference',
              sulfix='_ref')

#### Download Planet images

In [12]:
%%time
GetImageChips(download_image=planet_image,
              out_resolution=5,
              points=points,
              out_dir='../../data/images/3_planet',
              sulfix='_planet')

Process ForkPoolWorker-45:
Process ForkPoolWorker-44:
Process ForkPoolWorker-40:
Process ForkPoolWorker-50:
Process ForkPoolWorker-43:
Process ForkPoolWorker-29:
Process ForkPoolWorker-34:
Process ForkPoolWorker-39:
Process ForkPoolWorker-47:
Process ForkPoolWorker-35:
Process ForkPoolWorker-49:
Process ForkPoolWorker-31:
Process ForkPoolWorker-42:
Process ForkPoolWorker-36:
Process ForkPoolWorker-28:
Process ForkPoolWorker-26:
Process ForkPoolWorker-37:
Process ForkPoolWorker-32:
Process ForkPoolWorker-33:
Process ForkPoolWorker-48:
Process ForkPoolWorker-38:
Process ForkPoolWorker-27:
Process ForkPoolWorker-30:
Process ForkPoolWorker-41:
Process ForkPoolWorker-46:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback 

KeyboardInterrupt: 

  File "/opt/conda/lib/python3.8/multiprocessing/pool.py", line 125, in worker
    result = (True, func(*args, **kwds))
  File "/opt/conda/lib/python3.8/multiprocessing/pool.py", line 125, in worker
    result = (True, func(*args, **kwds))
  File "/opt/conda/lib/python3.8/multiprocessing/pool.py", line 125, in worker
    result = (True, func(*args, **kwds))
  File "/opt/conda/lib/python3.8/multiprocessing/pool.py", line 125, in worker
    result = (True, func(*args, **kwds))
  File "/opt/conda/lib/python3.8/multiprocessing/pool.py", line 125, in worker
    result = (True, func(*args, **kwds))
  File "/opt/conda/lib/python3.8/multiprocessing/pool.py", line 125, in worker
    result = (True, func(*args, **kwds))
  File "/opt/conda/lib/python3.8/multiprocessing/pool.py", line 125, in worker
    result = (True, func(*args, **kwds))
  File "/opt/conda/lib/python3.8/multiprocessing/pool.py", line 125, in worker
    result = (True, func(*args, **kwds))
  File "/opt/conda/lib/python3.8/multipr

#### Download Sentinel-1 images

In [ ]:
%%time
GetImageChips(download_image=s1_image,
              out_resolution=20,
              points=points,
              out_dir='../../data/images/5_sentinel1',
              sulfix='_s1')

#### Download Sentinel-2 images

In [ ]:
%%time
GetImageChips(download_image=sentinel_image,
              out_resolution=20,
              points=points,
              out_dir='../../data/images/7_sentinel2',
              sulfix='_s2')